In [1]:
import numpy as np
from numpy import fft

import ipywidgets as widgets
from ipywidgets import interact

import plotly.express as px
import plotly.graph_objects as go

import matplotlib.pyplot as plt

from IPython.display import display

### Parameters

In [2]:
cs = 343

f = 40e3  # carrier frequency
T = 1/f

wavelength = cs/f

# Linear phased array beamforming

In [3]:
def transducerPhase(n, d, phi):
  wavelength = cs/f

  return np.cos(n * 2 * np.pi / wavelength * d * np.sin(phi))

def transducer(n, d, phi, f, t):
  w = 2 * np.pi * f
  wavelength = cs/f
  
  return np.cos(w * t + n * 2 * np.pi / wavelength * d * np.sin(phi))


In [4]:
def angularEnergy(V):
  vE = np.abs(V)
  E = np.array(np.sum(vE, 1)).flatten()
  E_dB = 20 * np.log10(E/max(E))
  return E_dB

## 2 transducer

### Time signals

In [5]:
t = np.arange(-T, T, T/100)

phi_slider = widgets.FloatSlider(
    value=0,
    min=-np.pi,
    max=np.pi,
    step=0.01,
    description="Angle: ",
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.2f',
)

d_slider = widgets.FloatSlider(
    value=wavelength/2,
    min=0,
    max=wavelength,
    step=wavelength/100,
    description="Distance: ",
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.5f',
)

figT = go.FigureWidget()
figT.update_layout(
    autosize=False,
    width=800,
    height=500,
    margin=dict(
        l=40,
        r=30,
        b=40,
        t=30,
        pad=4
    ),
    xaxis_title="t in s",
    yaxis_title="Magnitude",
    yaxis=dict(range=[-2, 2])
)
figT.add_scatter(x=t, name="n: 0")
figT.add_scatter(x=t, name="n: 1")
figT.add_scatter(x=t, name="Sum")

@interact(phi=phi_slider, d=d_slider)
def plot2DBeamSignals(phi, d):
    E1 = transducer(0, d, phi, f, t)
    E2 = transducer(1, d, phi, f, t)
    E = E1 + E2
    
    figT.data[0].y = E1
    figT.data[1].y = E2
    figT.data[2].y = E
  
figT

interactive(children=(FloatSlider(value=0.0, continuous_update=False, description='Angle: ', max=3.14159265358…

FigureWidget({
    'data': [{'name': 'n: 0',
              'type': 'scatter',
              'uid': '22a998c0-4…

### Polar signal

In [6]:
N = 2
#vphi = np.transpose(np.matrix(np.arange(-np.pi, np.pi, np.pi/200)))
vphi = np.arange(-np.pi, np.pi, np.pi/200)

d_slider = widgets.FloatSlider(
    value=wavelength/2,
    min=0,
    max=wavelength,
    step=wavelength/100,
    description="Distance: ",
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.5f',
)

figP = go.FigureWidget()
figP.update_layout(
    autosize=False,
    width=800,
    height=500,
    margin=dict(
        l=40,
        r=30,
        b=40,
        t=30,
        pad=4
    ),
    polar = dict(radialaxis_range=[-30, 0])
)
figP.add_scatterpolar(theta=vphi)
polarP = figP.data[0]

@interact(d=d_slider)
def plot2DBeamPolar(d):
  E = np.zeros(len(vphi))
  for n in range(N):
    E = E + transducerPhase(n, d, vphi)

  E_dB = 20 * np.log10(1/N * np.abs(E))
    
  polarP.r = E_dB

figP


interactive(children=(FloatSlider(value=0.0042875, continuous_update=False, description='Distance: ', max=0.00…

FigureWidget({
    'data': [{'r': array([ 0.        , -0.00528813, -0.02115374, ..., -0.0476005 , -0.02115374,…

### Polar with FFT

In [7]:
N = 2
#vphi = np.transpose(np.matrix(np.arange(-np.pi, np.pi, np.pi/200)))
vphi = np.arange(-np.pi, np.pi, np.pi/200)
t = np.arange(0, T, T/100)

t_grid, vphi_grid = np.meshgrid(t, vphi)

d_slider = widgets.FloatSlider(
    value=wavelength/2,
    min=0,
    max=wavelength,
    step=wavelength/100,
    description="Distance: ",
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.5f',
)

figP = go.FigureWidget()
figP.update_layout(
    autosize=False,
    width=800,
    height=500,
    margin=dict(
        l=40,
        r=30,
        b=40,
        t=30,
        pad=4
    ),
    polar=dict(radialaxis_range=[-30, 0])
)
figP.add_scatterpolar(theta=vphi)
polarP = figP.data[0]


@interact(d=d_slider)
def plot2DBeamPolar(d):
  s = np.zeros([len(vphi), len(t)])

  for n in range(N):
    s = s + transducer(n, d, vphi_grid, f, t_grid)

  S = np.abs(fft.fft(s))
  E = np.sum(S, 1)
  E_dB = 20 * np.log10(E/np.max(E))

  polarP.r = E_dB

figP

interactive(children=(FloatSlider(value=0.0042875, continuous_update=False, description='Distance: ', max=0.00…

FigureWidget({
    'data': [{'r': array([-8.18689032e-04, -1.92865493e-15, -1.36217649e-03, ..., -6.48821601e-…

## N transducer
### Time signal

In [8]:
t = np.arange(-T, T, T/100)

phi_slider = widgets.FloatSlider(
    value=0,
    min=-np.pi,
    max=np.pi,
    step=0.01,
    description="Angle: ",
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.2f',
)

d_slider = widgets.FloatSlider(
    value=wavelength/2,
    min=0,
    max=wavelength,
    step=wavelength/100,
    description="Distance: ",
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.5f',
)

N_slider = widgets.IntSlider(
    value=2,
    min=1,
    max=20,
    step=1,
    description="Transducers: ",
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
)

figT2D = go.FigureWidget()
figT2D.update_layout(
    autosize=False,
    width=800,
    height=500,
    margin=dict(
        l=40,
        r=30,
        b=40,
        t=30,
        pad=4
    ),
    xaxis_title="t in s",
    yaxis_title="Magnitude"
)

@interact(phi=phi_slider, d=d_slider, N=N_slider)
def plot2DBeamSignals(phi, d, N):
    En = np.zeros([N, len(t)])
    
    figT2D.data = []
    figT2D.update_layout(yaxis=dict(range=[-N, N]))
    
    for n in range(N):
        En[n] = transducer(n, d, phi, f, t)
        figT2D.add_scatter(x=t, y=En[n], name=f"n: {n}")

    E = np.sum(En, 0)
    
    figT2D.add_scatter(x=t, y=E, name="Sum")


figT2D


interactive(children=(FloatSlider(value=0.0, continuous_update=False, description='Angle: ', max=3.14159265358…

FigureWidget({
    'data': [{'name': 'n: 0',
              'type': 'scatter',
              'uid': '2027a499-e…

### Polar signal

In [9]:
#vphi = np.transpose(np.matrix(np.arange(-np.pi, np.pi, np.pi/1000)))
vphi = np.arange(-np.pi, np.pi, np.pi/1000)

d_slider = widgets.FloatSlider(
    value=wavelength/2,
    min=0,
    max=2 * wavelength,
    step=wavelength/100,
    description="Distance: ",
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.5f',
)

N_slider = widgets.IntSlider(
    value=2,
    min=1,
    max=20,
    step=1,
    description="Transducers: ",
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
)

figP2D = go.FigureWidget()
figP2D.update_layout(
    autosize=False,
    width=800,
    height=500,
    margin=dict(
        l=40,
        r=30,
        b=40,
        t=30,
        pad=4
    ),
    polar=dict(radialaxis_range=[-30, 0])
)
figP2D.add_scatterpolar(theta=vphi)
polarP2D = figP2D.data[0]

@interact(d=d_slider, N=N_slider)
def plot2DBeamPolar(d, N):
  E = np.zeros(len(vphi))
  for n in range(N):
    E = E + transducerPhase(n, d, vphi)

  E_dB = 20 * np.log10(1/N * np.abs(E)) #20 * np.log10(np.abs(E)/np.max(np.abs(E)))

  polarP2D.r = E_dB

figP2D

interactive(children=(FloatSlider(value=0.0042875, continuous_update=False, description='Distance: ', max=0.01…

FigureWidget({
    'data': [{'r': array([ 0.        , -0.00021152, -0.00084609, ..., -0.0019037 , -0.00084609,…

### Polar with FFT

In [10]:
#vphi = np.transpose(np.matrix(np.arange(-np.pi, np.pi, np.pi/1000)))
vphi = np.arange(-np.pi, np.pi, np.pi/1000)
t = np.arange(0, T, T/100)

t_grid , vphi_grid = np.meshgrid(t, vphi)

d_slider = widgets.FloatSlider(
    value=wavelength/2,
    min=0,
    max=2 * wavelength,
    step=wavelength/100,
    description="Distance: ",
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.5f',
)

N_slider = widgets.IntSlider(
    value=2,
    min=1,
    max=20,
    step=1,
    description="Transducers: ",
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
)

figP2D = go.FigureWidget()
figP2D.update_layout(
    autosize=False,
    width=800,
    height=500,
    margin=dict(
        l=40,
        r=30,
        b=40,
        t=30,
        pad=4
    ),
    polar=dict(radialaxis_range=[-30, 0])
)
figP2D.add_scatterpolar(theta=vphi)
polarP2D = figP2D.data[0]

@interact(d=d_slider, N=N_slider)
def plot2DBeamPolar(d, N):
  s = np.zeros([len(vphi), len(t)])

  for n in range(N):
    s = s + transducer(n, d, vphi_grid, f, t_grid)

  S = np.abs(fft.fft(s))
  E = np.sum(S, 1)
  E_dB = 20 * np.log10(E/np.max(E))

  polarP2D.r = E_dB

figP2D

interactive(children=(FloatSlider(value=0.0042875, continuous_update=False, description='Distance: ', max=0.01…

FigureWidget({
    'data': [{'r': array([-0.00081869, -0.00068578, -0.00046385, ..., -0.00024787, -0.00046385,…

# 3D Array

In [11]:
def transducerPhase3D(n, m, d, phi, alpha):
  wavelength = cs/f

  return np.cos(2 * np.pi / wavelength * d * np.sqrt(np.power(n * np.sin(phi), 2) + np.power(m * np.sin(alpha), 2)))

def transducer3D(n, m, d, phi, alpha, f, t):
  w = 2 * np.pi * f
  wavelength = cs/f

  phase = 2 * np.pi / wavelength * d * np.sqrt(np.power(n * np.sin(phi), 2) + np.power(m * np.sin(alpha), 2))

  return np.cos(w * t + phase)

In [12]:
def angularEnergy3D(V):
  vE = np.power(np.abs(V), 2)
  E = np.array(np.sum(vE, 2))

  return E/np.max(E)

## N Transducer

### Time Signal

In [13]:
t = np.arange(-T, T, T/100)

phi_slider = widgets.FloatSlider(
    value=0,
    min=-np.pi,
    max=np.pi,
    step=0.01,
    description="Angle N: ",
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.2f',
)

alpha_slider = widgets.FloatSlider(
    value=0,
    min=-np.pi,
    max=np.pi,
    step=0.01,
    description="Angle M: ",
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.2f',
)

d_slider = widgets.FloatSlider(
    value=wavelength/2,
    min=0,
    max=wavelength,
    step=wavelength/100,
    description="Distance: ",
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.5f',
)

N_slider = widgets.IntSlider(
    value=2,
    min=1,
    max=20,
    step=1,
    description="N: ",
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
)

M_slider = widgets.IntSlider(
    value=2,
    min=1,
    max=20,
    step=1,
    description="M: ",
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
)

figT3D = go.FigureWidget()
figT3D.update_layout(
    autosize=False,
    width=800,
    height=500,
    margin=dict(
        l=40,
        r=30,
        b=40,
        t=30,
        pad=4
    ),
    xaxis_title="t in s",
    yaxis_title="Magnitude"
)


@interact(d=d_slider, phi=phi_slider, N=N_slider, alpha=alpha_slider, M=M_slider)
def plot3DBeamSignals(phi, alpha, d, N, M):
    En = np.zeros([N, M, len(t)])

    # plt.figure(figsize=[10, 4])
    figT3D.data = []
    figT3D.update_layout(yaxis=dict(range=[-N*M, N*M]))
    for n in range(N):
        for m in range(M):
            En[n, m] = transducer3D(n, m, d, phi, alpha, f, t)
            # plt.plot(t, En[n, m])
            figT3D.add_scatter(x=t, y=En[n,m], name=f"n: {n}, m: {m}", )

    E = np.sum(En, (0, 1))

    figT3D.add_scatter(x=t, y=E, name="Sum")

figT3D


interactive(children=(FloatSlider(value=0.0, continuous_update=False, description='Angle N: ', max=3.141592653…

FigureWidget({
    'data': [{'name': 'n: 0, m: 0',
              'type': 'scatter',
              'uid': '039b…

### Polar Signal

In [14]:
vphi = np.transpose(np.arange(-np.pi, np.pi, np.pi/200))
valpha = np.arange(-np.pi, np.pi, np.pi/200)

d_slider = widgets.FloatSlider(
    value=wavelength/2,
    min=0,
    max=2 * wavelength,
    step=wavelength/100,
    description="Distance: ",
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.5f',
)

N_slider = widgets.IntSlider(
    value=2,
    min=1,
    max=20,
    step=1,
    description="N: ",
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
)

M_slider = widgets.IntSlider(
    value=2,
    min=1,
    max=20,
    step=1,
    description="M: ",
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
)

figP3D = go.FigureWidget()
figP3D.add_surface()
figP3D.update_layout(
    autosize=False, 
    width=800, 
    height=500, 
    margin=dict(
        l=30,
        r=30,
        b=30,
        t=30,
        pad=4
    ),
    scene=dict(
        xaxis=dict(range=[-1, 1]),
        yaxis=dict(range=[-1, 1]),
        zaxis=dict(range=[-1, 1]),
    ),
)
surP3D = figP3D.data[0]

@interact(d=d_slider, N=N_slider, M=M_slider)
def plot3DBeamPolar(d, N, M):
    phi_grid, alpha_grid = np.meshgrid(vphi, valpha)

    E = np.zeros([len(vphi), len(valpha)])
    for n in range(N):
        for m in range(M):
            E = E + transducerPhase3D(n, m, d, phi_grid, alpha_grid)

    E_dB = 20 * np.log10(1/(N*M) * np.abs(E))

    X = np.cos(alpha_grid) * np.cos(phi_grid) * 1/(N*M) * E
    Y = np.cos(alpha_grid) * np.sin(phi_grid) * 1/(N*M) * E
    Z = np.sin(alpha_grid) * 1/(N*M) * E

    with figP3D.batch_update():
        surP3D.x = X
        surP3D.y = Y
        surP3D.z = Z

figP3D

interactive(children=(FloatSlider(value=0.0042875, continuous_update=False, description='Distance: ', max=0.01…

FigureWidget({
    'data': [{'type': 'surface',
              'uid': 'a5648bbd-1fa7-41c8-abd6-aecb7dcbcefe',
 …

### Polar with FFT

In [21]:
vphi = np.transpose(np.arange(-np.pi, np.pi, np.pi/100))
valpha = np.arange(-np.pi, np.pi, np.pi/100)
t = np.arange(0, T, T/10)

valpha_grid, vphi_grid, t_grid = np.meshgrid(valpha, vphi, t)
alpha_grid, phi_grid = np.meshgrid(valpha, vphi)

d_slider = widgets.FloatSlider(
    value=wavelength/2,
    min=0,
    max=2 * wavelength,
    step=wavelength/100,
    description="Distance: ",
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.5f',
)

N_slider = widgets.IntSlider(
    value=2,
    min=1,
    max=20,
    step=1,
    description="N: ",
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
)

M_slider = widgets.IntSlider(
    value=2,
    min=1,
    max=20,
    step=1,
    description="M: ",
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
)

figP3D = go.FigureWidget()
figP3D.add_surface()
figP3D.update_layout(
    autosize=False, 
    width=800, 
    height=500, 
    margin=dict(
        l=30,
        r=30,
        b=30,
        t=30,
        pad=4
    ),
    scene=dict(
        xaxis=dict(range=[-1, 1]),
        yaxis=dict(range=[-1, 1]),
        zaxis=dict(range=[-1, 1]),
    ),
)
surP3D = figP3D.data[0]

@interact(d=d_slider, N=N_slider, M=M_slider)
def plot3DBeamPolar(d, N, M):
    s = np.zeros([len(valpha), len(vphi), len(t)])
    for n in range(N):
        for m in range(M):
            s = s + transducer3D(n, m, d, vphi_grid, valpha_grid, f, t_grid)

    S = np.abs(fft.fft(s))
    E = np.sum(S, 2)

    X = np.cos(alpha_grid) * np.cos(phi_grid) * 1/np.max(E) * E
    Y = np.cos(alpha_grid) * np.sin(phi_grid) * 1/np.max(E) * E
    Z = np.sin(alpha_grid) * 1/np.max(E) * E

    with figP3D.batch_update():
        surP3D.x = X
        surP3D.y = Y
        surP3D.z = Z

figP3D

interactive(children=(FloatSlider(value=0.0042875, continuous_update=False, description='Distance: ', max=0.01…

FigureWidget({
    'data': [{'type': 'surface',
              'uid': '73179e70-2488-4b8c-9f66-d5740f5b3f30',
 …

In [16]:
vphi = np.transpose(np.arange(-np.pi, np.pi, np.pi/200))
valpha = np.arange(-np.pi, np.pi, np.pi/200)
t = np.arange(0, T, T/100)

N = 2
M = 2

valpha_grid, vphi_grid, t_grid = np.meshgrid(valpha, vphi, t)

s = np.zeros([len(valpha), len(vphi), len(t)])
for n in range(N):
    for m in range(M):
        s = s + transducer3D(n, m, wavelength/2, vphi_grid, valpha_grid, f, t_grid)

S = np.abs(fft.fft(s))
E = np.sum(S, 2)
E_dB = 20 * np.log10(E/np.max(E))


# Modulated Signal

In [17]:
def transducerMod(n, d, phi, f, fs, t, m, U0):
  w = 2 * np.pi * f
  ws = 2 * np.pi * fs
  wavelength = cs/f

  s = np.sin(ws * t)
  p = n * 2 * np.pi / wavelength * d * np.sin(phi)
  c = U0 * np.cos(w * (1 + m * s) * t + p)

  return c

In [18]:
fs = 1e3
Ts = 1/fs

m = 0.2

## Time

In [19]:
t = np.arange(-Ts, Ts, Ts/1000)

phi_slider = widgets.FloatSlider(
    value=0,
    min=-np.pi,
    max=np.pi,
    step=0.01,
    description="Angle: ",
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.2f',
)

d_slider = widgets.FloatSlider(
    value=wavelength/2,
    min=0,
    max=wavelength,
    step=wavelength/100,
    description="Distance: ",
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.5f',
)

N_slider = widgets.IntSlider(
    value=2,
    min=1,
    max=20,
    step=1,
    description="Transducers: ",
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
)

m_slider = widgets.FloatSlider(
    value=m,
    min=0,
    max=2,
    step=0.01,
    description="Modulation: ",
    orientation='horizontal',
    readout=True,
    readout_format=".2f",
)

figT2DM = go.FigureWidget()
figT2DM.update_layout(
    autosize=False,
    width=800,
    height=500,
    margin=dict(
        l=40,
        r=30,
        b=40,
        t=30,
        pad=4
    ),
    xaxis_title="t in s",
    yaxis_title="Magnitude"
)


@interact(phi=phi_slider, d=d_slider, N=N_slider, m=m_slider)
def plot2DModulatedBeamSignals(phi, d, N, m):
    En = np.zeros([N, len(t)])

    figT2DM.data = []
    figT2DM.update_layout(yaxis=dict(range=[-N, N]))

    for n in range(N):
        En[n] = transducerMod(n, d, phi, f, fs, t, m, 1)
        figT2DM.add_scatter(x=t, y=En[n], name=f"n: {n}")

    E = np.sum(En, 0)

    display(f"Frequency Range: ±{((f * m)/(1e3)):.2f}kHz")
    figT2DM.add_scatter(x=t, y=E, name="Sum")


figT2DM


interactive(children=(FloatSlider(value=0.0, continuous_update=False, description='Angle: ', max=3.14159265358…

FigureWidget({
    'data': [{'name': 'n: 0',
              'type': 'scatter',
              'uid': '9d2fc430-f…

# Polar

In [20]:
vphi = np.arange(-np.pi, np.pi, np.pi/1000)
t = np.arange(0, T, T/100)

t_grid, vphi_grid = np.meshgrid(t, vphi)

d_slider = widgets.FloatSlider(
    value=wavelength/2,
    min=0,
    max=2 * wavelength,
    step=wavelength/100,
    description="Distance: ",
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.5f',
)

N_slider = widgets.IntSlider(
    value=2,
    min=1,
    max=20,
    step=1,
    description="Transducers: ",
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
)

figP2DM = go.FigureWidget()
figP2DM.update_layout(
    autosize=False,
    width=800,
    height=500,
    margin=dict(
        l=40,
        r=30,
        b=40,
        t=30,
        pad=4
    ),
    polar=dict(radialaxis_range=[-30, 0])
)
figP2DM.add_scatterpolar(theta=vphi)
polarP2DM = figP2DM.data[0]


@interact(d=d_slider, N=N_slider)
def plot2DBeamPolar(d, N):
  s = np.zeros([len(vphi), len(t)])

  for n in range(N):
    s = s + transducer(n, d, vphi_grid, f, t_grid)

  S = np.abs(fft.fft(s))
  E = np.sum(S, 1)
  E_dB = 20 * np.log10(E/np.max(E))

  polarP2DM.r = E_dB


figP2DM


interactive(children=(FloatSlider(value=0.0042875, continuous_update=False, description='Distance: ', max=0.01…

FigureWidget({
    'data': [{'r': array([-0.00081869, -0.00068578, -0.00046385, ..., -0.00024787, -0.00046385,…